In [28]:
import ast
import pandas as pd

In [29]:
rows = []
with open('australian_user_reviews.json',encoding='MacRoman') as f: ## Cargamos y leemos los registros
  for line in f.readlines():
    rows.append(ast.literal_eval(line)) ## Linea por linea

df_reviews = pd.DataFrame(rows)

In [30]:
df_reviews.shape

(25799, 3)

In [31]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [32]:
#Explotamos la columna "reviews" del dataframe en un nuevo dataframe.
df_comenExpan = df_reviews.explode('reviews')

#Concatenamos el dataframe original con el nuevo dataframe creado
# y elimina la columna "reviews" del dataframe concatenado
df_reviews = pd.concat([df_comenExpan.drop(['reviews'], axis=1), df_comenExpan['reviews'].apply(pd.Series)], axis=1)

In [33]:
df_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


In [34]:
columns = ['user_id', 'posted', 'item_id','recommend','review']
df_reviews = df_reviews[columns]

In [35]:
df_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
0,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
0,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
1,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


In [36]:
df_names = pd.read_csv('id_names.csv')
df_names.head()

,id,app_name
0,10.0,Counter-Strike
1,20.0,Team Fortress Classic
2,30.0,Day of Defeat
3,40.0,Deathmatch Classic
4,50.0,Half-Life: Opposing Force


In [37]:
df_names['id'] = df_names['id'].astype(int)

In [38]:
#Renombramos para facilitar el merge.
df_reviews.rename(columns={'item_id': 'id'}, inplace=True)


In [39]:
#Eliminamos los nulos de la columna 'id'.
df_reviews = df_reviews.dropna(subset=['id'])


In [40]:
#Convertimos la columna 'id' en integers para facilitar el merge.
df_reviews['id'] = df_reviews['id'].astype(int)

In [41]:
#Hacemos el merge.
df_reviews = pd.merge(df_reviews, df_names, on='id', how='inner')
df_reviews.head()

,user_id,posted,id,recommend,review,app_name
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,Killing Floor
1,death-hunter,"Posted March 30, 2015.",1250,True,"Amazing, Non-stop action of blowing stuff to b...",Killing Floor
2,DJKamBer,"Posted July 12, 2013.",1250,True,"Compared to Left 4 Dead 2, this game REALLY gi...",Killing Floor
3,diego9031,"Posted August 13, 2015.",1250,True,Jogo ‚ô•‚ô•‚ô•‚ô•.,Killing Floor
4,76561198081962345,"Posted April 5, 2014.",1250,True,cara nas imagens esse jogo da pouco de medo ma...,Killing Floor


In [42]:
#Ordenamos el dataframe en base a la columna 'id'
df_reviews = df_reviews.sort_values(by='id')

In [43]:
#Extraigo los anios desde la columna 'posted'.
df_reviews['year'] = df_reviews['posted'].str.extract(r'(\d{4})')

In [44]:
#Imprimo los valores unicos para poder rellenar el nulo con el anio faltante de la serie
unique_years = df_reviews['year'].unique()
print(unique_years)

['2015' '2014' nan '2013' '2012' '2011' '2010']


In [45]:
#Lleno los nulos con el valor faltante de la serie '2016'
# asumiendo ese valor para no eliminar todas esa data que 
# no tiene anio especificado.
df_reviews['year'].fillna(2016, inplace=True)

In [46]:
#Convierto la columna 'year' en integers.
df_reviews['year'] = df_reviews['year'].astype(int)

In [47]:
#Imprimo para checkear
df_reviews.head()

,user_id,posted,id,recommend,review,app_name,year
36298,TheRealChris02,"Posted February 17, 2015.",10,True,Best old school game,Counter-Strike,2015
36289,pwnddumass,"Posted April 10, 2014.",10,True,Really good game for its time! if you have a t...,Counter-Strike,2014
36290,NicolasNic,"Posted June 21, 2014.",10,True,2012-2014: awsome classic game changing shoot...,Counter-Strike,2014
36291,Hype_R,Posted January 30.,10,True,Best God Damn Game Known To Man Kind,Counter-Strike,2016
36292,KILLERamateur,"Posted March 9, 2014.",10,True,I heard that when i was made it was a huge suc...,Counter-Strike,2014


In [48]:
df_reviews.dtypes

user_id      object
posted       object
id            int32
recommend    object
review       object
app_name     object
year          int32
dtype: object

In [49]:
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Descargamos los recursos de NLTK (solo es necesario hacerlo una vez)
nltk.download('vader_lexicon')

# Creamos la instancia del SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Creamos una función para realizar el análisis de sentimiento
def analyze_sentiment(text):

    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity

    vader_scores = sia.polarity_scores(text)
    compound_sentiment = vader_scores['compound']

    if compound_sentiment >= 0.05:
        return 2  # Positivo
    elif compound_sentiment <= -0.05:
        return 0  # Negativo
    else:
        return 1  # Neutral


# Aplicamos la función de análisis de sentimiento a la columna "review"
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analyze_sentiment)

# Mapeamos los valores numéricos a etiquetas de sentimiento
sentiment_mapping = {
    0: 'Negativo',
    1: 'Neutral',
    2: 'Positivo'
}
df_reviews['sentiment_analysis'] = df_reviews['sentiment_analysis'].map(sentiment_mapping)

# Imprimimos el DataFrame resultante
print(df_reviews)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Maverick\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                 user_id                     posted      id recommend  \
36298     TheRealChris02  Posted February 17, 2015.      10      True   
36289         pwnddumass     Posted April 10, 2014.      10      True   
36290         NicolasNic      Posted June 21, 2014.      10      True   
36291             Hype_R         Posted January 30.      10      True   
36292      KILLERamateur      Posted March 9, 2014.      10      True   
...                  ...                        ...     ...       ...   
48650  76561198075141715          Posted August 25.  521430      True   
47323        TfhuAWGscvg          Posted August 30.  521570      True   
47324  76561198071122396          Posted August 29.  521570      True   
48196           mikeyg74       Posted September 17.  521990      True   
48075           Limo1300       Posted September 15.  527340      True   

                                                  review  \
36298                               Best old school game   
362

In [50]:
#Guardo el archivo final en un '.csv'
df_reviews.to_csv('reviews.csv', index=False)